In [ ]:
from pymol import cmd
from rdkit import Chem
from rdkit.Chem import Draw, AllChem
import py3Dmol

from openbabel import pybel

import sys, os, random
sys.path.insert(1, 'utilities/')

from utils import get_inplace_rmsd, get_scaffold_based_conformers, getbox

import warnings
warnings.filterwarnings("ignore")

%config Completer.use_jedi = False

In [ ]:
os.chdir('test/Covalent_Docking/')

In [ ]:
cmd.fetch(code='6ZRU',type='pdb1')
cmd.select(name='Prot',selection='polymer.protein')
cmd.select(name='Boceprevir',selection='resn U5G')
cmd.save(filename='6ZRU_clean.pdb',format='pdb',selection='Prot')
cmd.save(filename='Boceprevir_ref.sdf',format='sdf',selection='Boceprevir')
cmd.delete('all')

In [ ]:
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open('6ZRU_clean.pdb','r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})

view.addModel(open('Boceprevir_ref.sdf','r').read(),format='sdf')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'greenCarbon','radius':0.2}})

view.zoomTo()
view.show()

In [ ]:
Boceprevir=Chem.MolFromSmiles('CC(C)(C)NC(=O)N[C@H](C(=O)N1C[C@H]2[C@@H]([C@H]1C(=O)N[C@@H](CC1CCC1)C(O)(S)C([NH3+])=O)C2(C)C)C(C)(C)C')
Boceprevir=Chem.AddHs(Boceprevir)
AllChem.EmbedMolecule(Boceprevir)
Boceprevir

In [ ]:
writer=Chem.SDWriter('Boce_prep.sdf')
writer.write(Boceprevir)
writer.close()

In [ ]:
get_scaffold_based_conformers(smiles='CC(C)(C)NC(=O)N[C@H](C(=O)N1C[C@H]2[C@@H]([C@H]1C(=O)N[C@@H](CC1CCC1)C(O)(S)C([NH3+])=O)C2(C)C)C(C)(C)C',anchor='S_atom.sdf',num_confs=100,rmsd_threshold=0.75,output='Confs.sdf')

In [ ]:
!../../bin/lepro_linux_x86 {'6ZRU_clean.pdb'}

os.rename('pro.pdb','6ZRU_clean_H.pdb')

In [ ]:
confs=Chem.SDMolSupplier('Confs.sdf')

In [ ]:
mol=confs[0]
for atom in mol.GetAtoms():
    atom.SetProp("atomNote", str(atom.GetIdx()+1))

Draw.MolToImage(mol, size=(800,500))

In [ ]:
x_out=Chem.SDWriter('jfjdfj.sdf')
for c in confs:
    ed=AllChem.EditableMol(c)
    ed.RemoveAtom(27)
    x_out.write(ed.GetMol())
x_out.close()

In [ ]:
cmd.load(filename='6ZRU_clean_H.pdb',format='pdb',object='prot') #Not needed but as reference of the system
cmd.load(filename='Boceprevir_ref.sdf',format='sdf',object='lig')

center,size=getbox(selection='resn Cys and resid 145',extending=6.0,software='vina')
cmd.delete('all')
print(center)
print(size)

In [ ]:

!../../bin/smina -r {'6ZRU_clean_H.pdb'} -l {'Boce_prep.sdf'} -o {'Smina.sdf'} --center_x {center['center_x']} --center_y {center['center_y']} --center_z {center['center_z']} --size_x {size['size_x']} --size_y {size['size_y']} --size_z {size['size_z']} --exhaustiveness 8 --num_modes 5 --custom_scoring {'Cov_ScoreFunction.txt'}

#candidates=Chem.SDWriter('Cans.sdf')
#results=Chem.SDMolSupplier('Smina.sdf')
#scaffold=Chem.SDMolSupplier('Adenine.sdf')[0]
#for res in results:
    #rmsd= get_inplace_rmsd (scaffold,res)
    #if float(res.GetProp('minimizedAffinity')) < 0:
        #res.SetProp('RMSDvsScaffold',str(rmsd))
        #candidates.write(res)
#candidates.close()

In [ ]:
!../../bin/smina --print_atom_types

In [ ]:
from rdkit.Chem import AllChem, Draw

In [ ]:
mol = Chem.MolFromSmiles('CC(C)(C)NC(=O)N[C@H](C(=O)N1C[C@H]2[C@@H]([C@H]1C(=O)N[C@@H](CC1CCC1)[C@@H](O)C([NH3+])=O)C2(C)C)C(C)(C)C',sanitize=True)
#mol=Chem.AddHs(mol)
AllChem.EmbedMolecule(mol)
mol

In [ ]:
for atom in mol.GetAtoms():
    atom.SetProp("atomNote", str(atom.GetIdx()+1))

Draw.MolToImage(mol, size=(800,500))

In [ ]:
f=Chem.rdGeometry.Point3D(0,0,0)

In [ ]:
list(f)

In [ ]:
coors = dict()

for a,b in amap:
    coors[a] = constrain.GetConformer().GetAtomPosition(b)

w = Chem.SDWriter(output)

mol.UpdatePropertyCache()
constrain.UpdatePropertyCache()

confs = AllChem.EmbedMultipleConfs(mol,
    numConfs=int(num_confs),
    coordMap=coors,
    pruneRmsThresh=0.75,
    useExpTorsionAnglePrefs=True,
    useBasicKnowledge=True)

In [ ]:
with open('ddd.smiles','r') as mols:
    for line in mols.readlines():
        mol=Chem.MolFromSmiles(line)
        print(mol)